This workbook is to create a general vector store with meta data category.

In [ ]:
# https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents_langchain_matching_engine.ipynb

In [1]:
!gsutil cp gs://vector_search_paramters/ecg_all_me.json ../vector_store_me_parameters/

Copying gs://vector_search_paramters/ecg_all_me.json...
/ [1 files][  384.0 B/  384.0 B]                                                
Operation completed over 1 objects/384.0 B.                                      


In [5]:
import sys

# Add the directory to sys.path
sys.path.append('/home/qianyucazelles/internal_knowledge_chatbot/src/utils')

In [3]:
from doc_to_vertex_search import *

# 2. Load pdfs

In [6]:
from loader_splitter import *

##### - load all dpo files

In [10]:
cnil_docs_unstructured, unloaded_pdfs, loaded_pdfs, other_files = process_folder(folder_path=f"../documents/ecg_dpo/CNIL",unstructure=True)

[2024-05-16 12:18:52] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE II - Principes _ CNIL.pdf
Using unstrucure loader
[2024-05-16 12:18:53] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE I - Dispositions générales.pdf
Using unstrucure loader
[2024-05-16 12:18:54] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE XI - Dispositions finales _ CNIL.pdf
Using unstrucure loader
[2024-05-16 12:18:55] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE V - Transferts de données à caractère personnel vers des pays tiers ou à des organisations internationales _ CNIL.pdf
Using unstrucure loader
[2024-05-16 12:18:57] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE VII - Coopération et cohérence _ CNIL.pdf
Using unstrucure loader
[2024-05-16 12:18:59] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE VI - Autorités de contrôle indépendantes _ CNIL.pdf
Using unstrucure loader
[2024-05-16 12:19:01] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE III - Droits de la personne concernée _ CNIL.pdf
Using unstrucure loader
[2024-05-16 12:

In [13]:
len(cnil_docs_unstructured)

11

In [11]:
len(unloaded_pdfs)

0

In [14]:
cnil_docs_structured, unloaded_pdfs, loaded_pdfs, other_files = process_folder(folder_path=f"../documents/ecg_dpo/CNIL",unstructure=False)

[2024-05-16 12:19:48] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE II - Principes _ CNIL.pdf
[2024-05-16 12:19:49] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE I - Dispositions générales.pdf
[2024-05-16 12:19:49] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE XI - Dispositions finales _ CNIL.pdf
[2024-05-16 12:19:50] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE V - Transferts de données à caractère personnel vers des pays tiers ou à des organisations internationales _ CNIL.pdf
[2024-05-16 12:19:50] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE VII - Coopération et cohérence _ CNIL.pdf
[2024-05-16 12:19:52] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE VI - Autorités de contrôle indépendantes _ CNIL.pdf
[2024-05-16 12:19:52] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE III - Droits de la personne concernée _ CNIL.pdf
[2024-05-16 12:19:53] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE VIII - Voies de recours, responsabilité et sanctions _ CNIL.pdf
[2024-05-16 12:19:54] Scanning ../documents/ecg_dpo/C

In [15]:
len(cnil_docs_structured)

101

In [16]:
len(unloaded_pdfs)

0

##### - load insurance file

In [21]:
!ls ../documents/ecg_assurance/'Contrats 2024'/MH/LLOYDS

'Conditions Générales MAMS RI 04.2021.pdf'
'DIP RI LLYODS 09 2022.pdf'
'HOMAIR VACANCES - Conditions Particulières au 01.10.2023.pdf'


In [23]:
insurance_structured, unloaded_pdfs, loaded_pdfs, other_files = process_folder(folder_path=f"../documents/ecg_assurance/Contrats 2024/MH/LLOYDS",unstructure=False)

[2024-05-16 12:26:00] Scanning ../documents/ecg_assurance/Contrats 2024/MH/LLOYDS/DIP RI LLYODS 09 2022.pdf
[2024-05-16 12:26:00] Scanning ../documents/ecg_assurance/Contrats 2024/MH/LLOYDS/Conditions Générales MAMS RI 04.2021.pdf
[2024-05-16 12:26:05] Scanning ../documents/ecg_assurance/Contrats 2024/MH/LLOYDS/HOMAIR VACANCES - Conditions Particulières au 01.10.2023.pdf


In [26]:
len(insurance_structured)

91

In [27]:
insurance_unstructured, unloaded_pdfs, loaded_pdfs, other_files = process_folder(folder_path=f"../documents/ecg_assurance/Contrats 2024/MH/LLOYDS",unstructure=True)

[2024-05-16 12:27:01] Scanning ../documents/ecg_assurance/Contrats 2024/MH/LLOYDS/DIP RI LLYODS 09 2022.pdf
Using unstrucure loader
[2024-05-16 12:27:02] Scanning ../documents/ecg_assurance/Contrats 2024/MH/LLOYDS/Conditions Générales MAMS RI 04.2021.pdf
Using unstrucure loader
[2024-05-16 12:27:10] Scanning ../documents/ecg_assurance/Contrats 2024/MH/LLOYDS/HOMAIR VACANCES - Conditions Particulières au 01.10.2023.pdf
Using unstrucure loader


In [29]:
len(insurance_unstructured)

3

In [28]:
len(unloaded_pdfs)

0

# 3. Split doc

In [87]:
# split articles to chunk
from langchain.text_splitter import RecursiveCharacterTextSplitter

def doc_to_splits( docs,
                  chunk_size ,
                  separators = ["\n\n", ".", "!", "?"],               
                  ):
    '''
    Function to split docs into chunks with chunk idx
    '''
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size*0.2),
        separators=separators
    )

    # add chunk number
    doc_splits_list = []
    for doc in docs:
        doc_splits = text_splitter.split_documents([doc])

        # Add chunk number to metadata
        for idx, split in enumerate(doc_splits):
            split.metadata["document_name"] = split.metadata['source'].split('/')[-1]
            split.metadata["chunk"] = f'{idx+1}/{len(doc_splits)}'
        doc_splits_list+=doc_splits

    return doc_splits_list

In [33]:
def get_docs_min_max_token_length(docs):
    '''
    Function to get min and max token length of a list of langchain docs
    '''
    min_nb = 1000
    min_idx = 0
    max_nb = 0
    max_idx = 0
    for _, doc in enumerate(docs):
        nb_ = nb_tokens(doc.page_content)
        if nb_>max_nb:
            max_nb = nb_
            max_idx = _
        if nb_<min_nb:
            min_nb = nb_
            min_idx = _
    return {min_idx:min_nb,max_idx:max_nb}

In [34]:
small_chunk = 500

In [35]:
medium_chunk = 2000

In [36]:
large_chunk = 4000

In [67]:
def add_metadata_label(docs, key_name, key_value):
    for _, doc in enumerate(docs):
        doc.metadata[key_name] = key_value
    return docs

In [74]:
def get_splits_w_labels(docs,separators,department,category,loader_method,chunk_size):
    splits = doc_to_splits(docs,
                      separators = separators,
                      chunk_size= chunk_size   
                      )
    splits = add_metadata_label(splits, 'department', department)
    splits = add_metadata_label(splits, 'category', category)
    splits = add_metadata_label(splits, 'loader_method', loader_method)
    splits = add_metadata_label(splits, 'chunk_size', chunk_size)
    return splits

#### - CNIL Chunks

##### - stuctured

In [60]:
# cnil_docs_structured[0]

In [59]:
# cnil_docs_structured[58]

In [49]:
cnil_docs_structured_separators =  ["\n\n", ".", "!", "?",";","\n"]

In [88]:
cnil_s_small =  get_splits_w_labels(cnil_docs_structured,
                                    cnil_docs_structured_separators,
                                    department = 'dpo',
                                    category ='cnil',
                                    loader_method = 'structured',
                                    chunk_size = small_chunk)

In [90]:
cnil_s_medium =  get_splits_w_labels(cnil_docs_structured,
                                    cnil_docs_structured_separators,
                                    department = 'dpo',
                                    category ='cnil',
                                    loader_method = 'structured',
                                    chunk_size = medium_chunk)

In [91]:
cnil_s_large =  get_splits_w_labels(cnil_docs_structured,
                                    cnil_docs_structured_separators,
                                    department = 'dpo',
                                    category ='cnil',
                                    loader_method = 'structured',
                                    chunk_size = large_chunk)

In [92]:
for splits in [cnil_s_small, cnil_s_medium, cnil_s_large ]:
    print(len(splits))

787
198
104


##### - unstructured

In [94]:
unstructured_separators = ["\n\nArticle","\n\n", ".", "!", "?"]

In [95]:
cnil_unstructured_small =  get_splits_w_labels(cnil_docs_unstructured,
                                    unstructured_separators,
                                    department = 'dpo',
                                    category ='cnil',
                                    loader_method = 'unstructured',
                                    chunk_size = small_chunk)

In [96]:
cnil_unstructured_medium =  get_splits_w_labels(cnil_docs_unstructured,
                                    unstructured_separators,
                                    department = 'dpo',
                                    category ='cnil',
                                    loader_method = 'unstructured',
                                    chunk_size = medium_chunk)

In [98]:
cnil_unstructured_large =  get_splits_w_labels(cnil_docs_unstructured,
                                    unstructured_separators,
                                    department = 'dpo',
                                    category ='cnil',
                                    loader_method = 'unstructured',
                                    chunk_size = large_chunk)

In [99]:
for splits in [cnil_unstructured_small, cnil_unstructured_medium, cnil_unstructured_large ]:
    print(len(splits))

712
175
90


##### - Insurance

##### - structured

In [110]:
separators = ["\n \n", "  \n",".", "!", "?"]

In [111]:
# insurance_structured[78]

In [112]:
insurance_s_small =  get_splits_w_labels(insurance_structured,
                                    separators,
                                    department = 'insurance',
                                    category ='mh',
                                    loader_method = 'structured',
                                    chunk_size = small_chunk)

In [116]:
len(insurance_s_small)

700

In [113]:
insurance_s_medium =  get_splits_w_labels(insurance_structured,
                                    separators,
                                    department = 'insurance',
                                    category ='mh',
                                    loader_method = 'structured',
                                    chunk_size = medium_chunk)

In [115]:
len(insurance_s_medium)

191

In [114]:
insurance_s_large=  get_splits_w_labels(insurance_structured,
                                    separators,
                                    department = 'insurance',
                                    category ='mh',
                                    loader_method = 'structured',
                                    chunk_size = large_chunk)

In [117]:
len(insurance_s_large)

91

##### - unstructured

In [122]:
separators = ["\n\n", "\n",".", "!", "?"]

In [123]:
# insurance_unstructured[1]

In [124]:
insurance_unstructured_small =  get_splits_w_labels(insurance_unstructured,
                                    separators,
                                    department = 'insurance',
                                    category ='mh',
                                    loader_method = 'unstructured',
                                    chunk_size = small_chunk)

In [125]:
len(insurance_unstructured_small)

667

In [126]:
insurance_unstructured_medium =  get_splits_w_labels(insurance_unstructured,
                                    separators,
                                    department = 'insurance',
                                    category ='mh',
                                    loader_method = 'unstructured',
                                    chunk_size = medium_chunk)

In [127]:
len(insurance_unstructured_medium)

151

In [128]:
insurance_unstructured_large=  get_splits_w_labels(insurance_unstructured,
                                    separators,
                                    department = 'insurance',
                                    category ='mh',
                                    loader_method = 'unstructured',
                                    chunk_size = large_chunk)

In [129]:
len(insurance_unstructured_large)

74

# 4. Add to vector store

In [130]:
from langchain_google_vertexai import (
    VertexAIEmbeddings,
    VectorSearchVectorStore
)

In [131]:
# https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings
embeddings = VertexAIEmbeddings(model_name="textembedding-gecko-multilingual@001")

##### - cnil large 

In [132]:
!gsutil cp gs://vector_search_paramters/ecg_all_me.json ../vector_store_me_parameters/

Copying gs://vector_search_paramters/ecg_all_me.json...
/ [1 files][  384.0 B/  384.0 B]                                                
Operation completed over 1 objects/384.0 B.                                      


In [133]:
parameters = get_me_parameters('../vector_store_me_parameters/ecg_all_me.json')

In [134]:
vector_store = get_vector_store(parameters, embeddings)

In [137]:
all_splits = []
for splits in [
    insurance_unstructured_large,insurance_unstructured_medium, insurance_unstructured_small,
    insurance_s_large, insurance_s_medium, insurance_s_small,
    cnil_s_large, cnil_s_medium,cnil_s_small,
    cnil_unstructured_large,cnil_unstructured_medium,cnil_unstructured_small
]:
    all_splits.extend(splits)

In [138]:
len(all_splits)

3940

In [145]:
# Store docs as embeddings in Matching Engine index
# It may take a while since API is rate limited
texts = [doc.page_content for doc in all_splits]
metadatas = [doc.metadata for doc in all_splits]

for i in range(0, len(texts), 1000):
    vector_store.add_texts(texts[i : i + 1000], metadatas[i : i + 1000])

Upserting datapoints MatchingEngineIndex index: projects/500033913879/locations/europe-west1/indexes/2976325199831498752
MatchingEngineIndex index Upserted datapoints. Resource name: projects/500033913879/locations/europe-west1/indexes/2976325199831498752
Upserting datapoints MatchingEngineIndex index: projects/500033913879/locations/europe-west1/indexes/2976325199831498752
MatchingEngineIndex index Upserted datapoints. Resource name: projects/500033913879/locations/europe-west1/indexes/2976325199831498752
Upserting datapoints MatchingEngineIndex index: projects/500033913879/locations/europe-west1/indexes/2976325199831498752
MatchingEngineIndex index Upserted datapoints. Resource name: projects/500033913879/locations/europe-west1/indexes/2976325199831498752
Upserting datapoints MatchingEngineIndex index: projects/500033913879/locations/europe-west1/indexes/2976325199831498752
MatchingEngineIndex index Upserted datapoints. Resource name: projects/500033913879/locations/europe-west1/inde